In [21]:
# test calibration using scipy.optimize


In [22]:
import unittest

import numpy as np
import scipy.optimize as opt

from openptv_python.calibration import (
    Calibration,
)
from openptv_python.imgcoord import image_coordinates
from openptv_python.orientation import (
    external_calibration,
)
from openptv_python.parameters import ControlPar, OrientPar, VolumePar, read_control_par
from openptv_python.tracking_frame_buf import TargetArray
from openptv_python.trafo import arr_metric_to_pixel
from openptv_python.imgcoord import img_coord
from openptv_python.trafo import pixel_to_metric
from openptv_python.tracking_frame_buf import Target


In [23]:
class TestGradientDescent(unittest.TestCase):
    # Based on the C tests in liboptv/check_orientation.c

    def setUp(self):
        control_file_name = "testing_folder/corresp/control.par"
        # self.control = ControlPar(4)
        self.control = read_control_par(control_file_name)

        self.orient_par_file_name = "testing_folder/corresp/orient.par"
        self.orient_par = OrientPar().from_file(self.orient_par_file_name)

        self.cal = Calibration().from_file(
            "testing_folder/calibration/cam1.tif.ori",
            "testing_folder/calibration/cam1.tif.addpar",
        )
        self.orig_cal = Calibration().from_file(
            "testing_folder/calibration/cam1.tif.ori",
            "testing_folder/calibration/cam1.tif.addpar",
        )

        
    def test_external_calibration(self):
        """External calibration using clicked points."""
        ref_pts = np.array(
            [
                [-40.0, -25.0, 8.0],
                [40.0, -15.0, 0.0],
                [40.0, 15.0, 0.0],
                [40.0, 0.0, 8.0],
            ]
        )

        # Fake the image points by back-projection
        targets = arr_metric_to_pixel(
            image_coordinates(ref_pts, self.cal, self.control.mm),
            self.control,
        )

        # Jigg the fake detections to give raw_orient some challenge.
        targets[:, 1] -= 0.1

        self.assertTrue(external_calibration(self.cal, ref_pts, targets, self.control))
        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=3
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
    )

In [24]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_external_calibration (__main__.TestGradientDescent)
External calibration using clicked points. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.014s

OK


Coefficients (beta): [ 1.80534911e-04 -6.77499328e-05 -6.52416361e-04 -1.77426081e-05
 -1.31470856e-07  4.02517087e-06]
Residuals: []
rank: 6
singular_values: None


In [25]:
control_file_name = "testing_folder/corresp/control.par"
control = read_control_par(control_file_name)

orient_par_file_name = "testing_folder/corresp/orient.par"
orient_par = OrientPar().from_file(orient_par_file_name)

cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar",
)
orig_cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar")

In [26]:
ref_pts = np.array(
    [
        [-40.0, -25.0, 8.0],
        [40.0, -15.0, 0.0],
        [40.0, 15.0, 0.0],
        [40.0, 0.0, 8.0],
    ]
)

# Fake the image points by back-projection
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, control.mm),
    control,
)

cal.set_pos(np.array([0, 0, 100]))
cal.set_angles(np.array([0, 0, 0]))

# Jigg the fake detections to give raw_orient some challenge.
targets[:, 1] -= 0.1

In [27]:
targs = [Target() for _ in targets]

for ptx, pt in enumerate(targets):
    targs[ptx].x = pt[0]
    targs[ptx].y = pt[1]

In [40]:


def residual(calibration_array, ref_pts, targs, control, cc):
    # print(calibration_array)
    # print(ref_pts)
    # print(targs)
    # print(control)
    # print(calibration_array)
    
    c = Calibration()
    c.set_pos(calibration_array[:3])
    c.set_angles(calibration_array[3:])
    c.int_par.cc = cc
    c.update_rotation_matrix()
    
    
    # print(f"{c.get_pos()=}")
                
    residual = 0
    for i in range(len(targs)):
        xc, yc = pixel_to_metric(targs[i].x, targs[i].y, control)
        # print(f"{xc=}, {yc=} mm")
        
        xp, yp = img_coord(ref_pts[i], c, control.mm)
        # print(f"{xp=}, {yp=} mm")
        residual += ((xc - xp)**2 + (yc - yp)**2)
        
        # print(f"{residual=}")
        
    return residual

In [41]:
x0 = np.hstack([cal.get_pos(), cal.get_angles()])
cc = orig_cal.int_par.cc

sol = opt.minimize(residual, x0, args=(ref_pts, targs, control, cc), method='Nelder-Mead', tol=1e-6)

In [43]:
print(f"{sol.x=}")
print(sol.x - np.hstack([orig_cal.get_pos(), orig_cal.get_angles()]))

sol.x=array([-3.79876331e+00,  3.77309709e+01,  4.03575316e+02, -8.61982784e-02,
       -6.33837589e-02,  5.14334685e-02])
[-1.09061963e+02 -6.50148291e+01 -3.06884099e-01  1.52130822e-01
 -3.07664759e-01 -3.82423146e-03]


In [39]:
print( residual(np.hstack([orig_cal.get_pos(), orig_cal.get_angles()]), ref_pts, targs, control, orig_cal.int_par.cc))

93.58853214427855
